In [1]:
!pip3 install torch
!pip3 install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 4.4 MB/s eta 0:00:0000:0100:01m
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 3.8 MB/s eta 0:00:0000:01
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.2 MB/s eta 0:00:0000:0100:010m
  Using cached kiwisolver-1.4.4-cp311-cp311-macosx_11_0_arm64.whl (63 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [175]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [176]:
names_list = open('names.txt','r').read().split('\n')

In [177]:
print(names_list[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [178]:
char_to_int = {}
int_to_char = {}

char_to_int['.'] = 0
int_to_char[0] = '.'

for char_num in range(ord('a'),ord('z')+1):
    integer_representation = char_num-ord('a')+1
    char_to_int[chr(char_num)] = integer_representation
    int_to_char[integer_representation] = chr(char_num)


In [244]:
#hyperparameters

embed_size = 7

state_size = 40

hidden_size = 200

vocab_size = 28

In [245]:
import math
char_vector_size = 2
input_contexts , input_labels = [[] for _ in range(5)], [[] for _ in range(5)]

for name in names_list:
    log_padded_length = math.ceil(math.log(len(name))/math.log(2))
    padded_length = int(2**log_padded_length)
    context = [char_to_int[char] for char in name]  + [0] + [vocab_size-1]*(padded_length-len(name)) 
    input_contexts[log_padded_length].append(context)
#   instead of using labels, we can compare the result to the character vectors to find a likely match

In [246]:
freqs = [0 for i in range(vocab_size-1)]

for log_len in range(1,5):
    contexts = input_contexts[log_len]
    for c in contexts:
        for int_char in c:
            if int_char<vocab_size-1:
                freqs[int_char] += 1
                
freqs = torch.tensor(freqs+[0],dtype=torch.float64)
freqs/=freqs.sum()

freqs

tensor([0.1404, 0.1485, 0.0116, 0.0155, 0.0241, 0.0895, 0.0040, 0.0084, 0.0334,
        0.0776, 0.0127, 0.0221, 0.0612, 0.0291, 0.0803, 0.0348, 0.0045, 0.0012,
        0.0557, 0.0355, 0.0244, 0.0137, 0.0113, 0.0041, 0.0031, 0.0428, 0.0105,
        0.0000], dtype=torch.float64)

In [247]:
word = input_contexts[2][1]

In [248]:
# parameters
all_char_embeds = torch.randn((vocab_size,embed_size))

all_char_embeds[vocab_size-1] = 10**10

w_eh = torch.randn((embed_size, hidden_size)) * 0.5

w_sh = torch.randn((state_size, hidden_size)) * 0.5
w_so = torch.randn((state_size, embed_size)) *0.5

w_hs = torch.randn((hidden_size, state_size)) *0.5

b_h = torch.zeros((hidden_size,))
b_s = torch.zeros((state_size,))
b_o = torch.zeros((embed_size,))

init_state = 2*torch.sigmoid(torch.randn((state_size,)))

network_params = [init_state,w_eh,w_sh,w_so,w_hs,b_h,b_s,b_o,all_char_embeds]

for p in network_params:
    p.requires_grad = True

In [249]:
def process_state_and_embed(states , embeds, hiddens, outputs, idx):
    hiddens[idx] = 2*torch.sigmoid((states[idx] @ w_sh) + (embeds[idx] @ w_eh) + b_h) -1
    states[idx+1] = 2*torch.sigmoid(hiddens[idx] @ w_hs + b_s) - 1
    outputs[idx] = states[idx] @ w_so + b_o

In [250]:
batch_size = 32

for i in range(len(input_contexts)):
    input_contexts[i] = torch.tensor(input_contexts[i])

In [251]:
loss_tot = 0

loops = 100000
check_every = 500

# training loop
for i in range(loops):
    log_idx = random.randint(1,4)
#     log_idx = 4
    batch_indices = torch.randint(0,input_contexts[log_idx].shape[0],(batch_size,))

    batch = input_contexts[log_idx][batch_indices]

    word_length = len(batch[0])

    embeds = torch.transpose(all_char_embeds[batch],0,1)
    
    factors = torch.where(freqs > 0,1/(freqs+1e-18),0)
    
    factors[:-1] /= factors[:-1].mean()
    
    adjustment_factors = torch.transpose(factors[batch],0,1)
    
    adjustment_factors = adjustment_factors.view((word_length,batch_size,1))
    
    adjustment_factors[:-1] /= adjustment_factors[:-1].mean()
#     print(adjustment_factors , embeds.shape)

    states = [init_state] + [torch.zeros((batch_size,state_size,)) for _ in range(word_length-1)]

    hiddens = [torch.zeros((batch_size,embed_size,)) for _ in range(word_length-1)]

    outputs = torch.zeros((word_length,batch_size,embed_size))

    for j in range(word_length-1):
        process_state_and_embed(states,embeds ,hiddens,outputs,j)

    outputs[-1] = states[-1] @ w_so + b_o
    
    distances = (torch.cdist(outputs,all_char_embeds[:-1]) ** 2).sum(2,keepdim=True)#is actually distances squared
        
    filtered = torch.where(embeds != 1e10, ((outputs-embeds)**2 / distances), 0)
    
    ones = torch.where(embeds != 1e10, 1, 0)
    
    loss = (filtered*adjustment_factors).sum()
    
    loss.backward()
    
    for p in network_params:
        p.data -= 1e-3*p.grad
        p.grad = None
        
    loss_tot+= loss/ adjustment_factors.sum()
    if i % check_every==0 and i>0:
        print(f"{i} of {loops}: " , loss_tot / (check_every))
        loss_tot = 0
#     print('o',outputs)
#     print('e',embeds)
#     print('wl',word_length)

500 of 100000:  tensor(0.0367, dtype=torch.float64, grad_fn=<DivBackward0>)
1000 of 100000:  tensor(0.0364, dtype=torch.float64, grad_fn=<DivBackward0>)
1500 of 100000:  tensor(0.0361, dtype=torch.float64, grad_fn=<DivBackward0>)
2000 of 100000:  tensor(0.0358, dtype=torch.float64, grad_fn=<DivBackward0>)
2500 of 100000:  tensor(0.0356, dtype=torch.float64, grad_fn=<DivBackward0>)
3000 of 100000:  tensor(0.0355, dtype=torch.float64, grad_fn=<DivBackward0>)
3500 of 100000:  tensor(0.0352, dtype=torch.float64, grad_fn=<DivBackward0>)
4000 of 100000:  tensor(0.0352, dtype=torch.float64, grad_fn=<DivBackward0>)
4500 of 100000:  tensor(0.0350, dtype=torch.float64, grad_fn=<DivBackward0>)
5000 of 100000:  tensor(0.0346, dtype=torch.float64, grad_fn=<DivBackward0>)
5500 of 100000:  tensor(0.0347, dtype=torch.float64, grad_fn=<DivBackward0>)
6000 of 100000:  tensor(0.0344, dtype=torch.float64, grad_fn=<DivBackward0>)
6500 of 100000:  tensor(0.0342, dtype=torch.float64, grad_fn=<DivBackward0>)


53500 of 100000:  tensor(0.0255, dtype=torch.float64, grad_fn=<DivBackward0>)
54000 of 100000:  tensor(0.0255, dtype=torch.float64, grad_fn=<DivBackward0>)
54500 of 100000:  tensor(0.0254, dtype=torch.float64, grad_fn=<DivBackward0>)
55000 of 100000:  tensor(0.0251, dtype=torch.float64, grad_fn=<DivBackward0>)
55500 of 100000:  tensor(0.0254, dtype=torch.float64, grad_fn=<DivBackward0>)
56000 of 100000:  tensor(0.0251, dtype=torch.float64, grad_fn=<DivBackward0>)
56500 of 100000:  tensor(0.0251, dtype=torch.float64, grad_fn=<DivBackward0>)
57000 of 100000:  tensor(0.0248, dtype=torch.float64, grad_fn=<DivBackward0>)
57500 of 100000:  tensor(0.0250, dtype=torch.float64, grad_fn=<DivBackward0>)
58000 of 100000:  tensor(0.0253, dtype=torch.float64, grad_fn=<DivBackward0>)
58500 of 100000:  tensor(0.0246, dtype=torch.float64, grad_fn=<DivBackward0>)
59000 of 100000:  tensor(0.0248, dtype=torch.float64, grad_fn=<DivBackward0>)
59500 of 100000:  tensor(0.0253, dtype=torch.float64, grad_fn=<D

In [242]:
def predict(word):
    word_length = len(word)

    embeds = all_char_embeds[word]

    states = [init_state]+[torch.zeros((state_size,)) for _ in range(word_length-1)]

    hiddens = [torch.zeros((embed_size,)) for _ in range(word_length-1)]

    outputs = torch.zeros((word_length,embed_size))

    for i in range(word_length-1):
        process_state_and_embed(states,embeds ,hiddens,outputs,i)

    return states[-1] @ w_so + b_o


def get_next(text):
    word = [char_to_int[c] for c in text] +[0]

    pred = predict(word)

    pred = pred.view(1,embed_size)
    dists = torch.cdist(pred,all_char_embeds[:-1])
    dists = dists.view(-1)
    dists = torch.exp(-7*dists)
    dists/=dists.sum()
    sampled = (torch.multinomial(dists,1)).item()
    return int_to_char[sampled]

In [243]:
start_st = ''

for _ in range(100):
    st = start_st
    curr = 'a'
    while curr!='.':
        curr = get_next(st)
        st+=curr
    print(st)


chuvtyixa.
puxai.
dhyi.
depey.
fhyvih.
wyyii.
cosuthay.
dhashay.
khixiyyen.
chuvriiq.
pugyae.
phkavbqpxasya.
mhzawayi.
polmi.
fhistya.
pudvhpya.
puxnai.
shetzi.
whuvhian.
mhqevici.
chuvgyaxy.
whtjmvvoie.
cottukhlvi.
gsitviy.
drarviy.
mafujey.
fyaa.
xtivey.
dtikcya.
mhmecana.
sopodayikh.
dya.
phpozkeviy.
catqateh.
kuiy.
warqin.
peftia.
pagfy.
mhvixje.
bhyka.
quyhi.
qtiaiy.
dhinayn.
xgavyi.
whuqmyia.
phjuvvqgjubnii.
barsevuvn.
bartioii.
gpyyay.
kuaa.
qayvfya.
phmwaga.
kuyy.
kuaa.
whumlae.
fhyxay.
koqzia.
bunhovehapeh.
tttpdhiy.
gcishiy.
phlehin.
cedtagya.
kuannae.
xehy.
chyviniiy.
luba.
gdythan.
luxa.
cepriivmey.
faxttan.
dsacwih.
khedeynwvxya.
wurbyn.
wrebci.
gwypavetnhn.
dsicvuia.
grhoxiiie.
khynh.
doay.
pyyn.
whuqdi.
cerpin.
zeruy.
wevuviuq.
fuanny.
wyya.
phsjsqveraoy.
chuqzyyya.
qncwvy.
chqyy.
dhyy.
musqii.
batregivquii.
ghewmay.
basnyay.
qyan.
cejhycaa.
zagay.
jhadhova.
gjahyhaiy.


In [165]:
((all_char_embeds[char_to_int['y']]-all_char_embeds[char_to_int['j']])**2).sum()

tensor(35.6892, grad_fn=<SumBackward0>)

In [218]:
all_char_embeds

tensor([[ 1.2277e+01,  3.9978e+00, -2.0790e+00, -6.4634e+00, -9.7122e-01,
          9.1355e+00],
        [ 6.2265e-02, -4.4034e-01, -6.3207e-02, -4.7214e-01,  9.0608e-01,
          8.7690e-01],
        [-7.4045e-01, -2.3842e-01,  1.1262e-01,  7.2173e-01, -3.0205e-01,
         -8.4768e-01],
        [-6.0282e-01, -1.2161e-01,  1.5330e-01,  5.2217e-01, -2.4655e-01,
         -7.8859e-01],
        [-7.0304e-01, -9.2246e-03,  3.6205e-01,  5.7091e-01, -2.2912e-01,
         -9.5274e-01],
        [ 1.0702e-01, -8.1980e-01, -1.4485e-01, -8.2107e-02,  1.1554e+00,
          7.8707e-01],
        [-7.8850e-01, -1.9774e-02, -7.0644e-03,  5.9431e-01, -2.7647e-01,
         -9.7013e-01],
        [-6.2197e-01,  1.8999e-01,  1.1287e-01,  4.8376e-01, -4.2226e-01,
         -6.9665e-01],
        [ 2.9039e-01, -5.6543e-01,  2.5713e-01,  5.0401e-02,  1.7475e-01,
          4.8427e-01],
        [ 6.7046e-01, -9.4411e-01, -4.5649e-01,  5.3217e-01,  1.4235e+00,
          1.0210e+00],
        [-1.0164e+00, -1.7325e